In [23]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from scipy.stats import multinomial
from statistics import mean
from math import log, e
import scipy.stats as stats


In [24]:
#reading in the data 
heart = pd.read_csv("heart.csv")
heart

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,0.0,1.0,1.0,1.0,18.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [25]:
#contigency table for fused variable: hd + stroke
data = pd.crosstab([heart.HeartDiseaseorAttack],heart.Stroke, margins = True)
data


Stroke,0.0,1.0,All
HeartDiseaseorAttack,,,
0.0,223432,6355,229787
1.0,19956,3937,23893
All,243388,10292,253680


In [26]:
#hd+stroke/gen health
data = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.GenHlth, margins = True)
data

GenHlth                        1.0    2.0    3.0    4.0    5.0     All
HeartDiseaseorAttack Stroke                                           
0.0                  0.0     43929  83666  65579  23160   7098  223432
                     1.0       354   1290   2153   1682    876    6355
1.0                  0.0       925   3754   6894   5431   2952   19956
                     1.0        91    374   1020   1297   1155    3937
All                          45299  89084  75646  31570  12081  253680

In [27]:
#high bp/high chol
data = pd.crosstab([heart.HighBP],heart.HighChol, margins = True)
data

HighChol,0.0,1.0,All
HighBP,,,
0.0,101920,42931,144851
1.0,44169,64660,108829
All,146089,107591,253680


In [28]:
#fused covariate against fixed response variable: hd_stroke against BP+HighChol
data_crosstab = pd.crosstab([heart.HeartDiseaseorAttack,heart.Stroke], 
                            [heart.HighBP,heart.HighChol],
                                margins = True)
data_crosstab

HighBP                          0.0           1.0            All
HighChol                        0.0    1.0    0.0    1.0        
HeartDiseaseorAttack Stroke                                     
0.0                  0.0      97968  38989  38448  48027  223432
                     1.0       1076    853   1457   2969    6355
1.0                  0.0       2489   2738   3548  11181   19956
                     1.0        387    351    716   2483    3937
All                          101920  42931  44169  64660  253680

In [29]:

contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.GenHlth, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)



Conditional Entropy: [1.77856531 1.57015622 1.53776851 1.22781048 1.79175947]


In [30]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.MentHlth, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [3.46417028 3.30802443 3.39773663 3.27050987 3.4657359 ]


In [31]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.Education, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [1.94399212 1.90043054 1.90339072 1.79032845 1.94591015]


In [32]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.Income, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [2.19513562 2.11226409 2.15297891 2.02363124 2.19722458]


In [33]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.Age, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [2.63392152 2.36168013 2.23480688 2.20829883 2.63905733]


In [34]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.HighBP, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [1.09609572 1.00613782 0.97543847 0.90815697 1.09861229]


In [35]:
contingency_table = pd.crosstab([heart.HeartDiseaseorAttack, heart.Stroke],heart.DiffWalk, margins = True)

# Calculate the conditional probability distribution
conditional_prob_dist = contingency_table / contingency_table.sum()

# Normalize the conditional probability distribution
normalized_conditional_prob_dist = conditional_prob_dist.div(conditional_prob_dist.sum(axis=1), axis=0)

# Calculate the conditional entropy
conditional_entropy = stats.entropy(normalized_conditional_prob_dist, axis=1)

print('Conditional Entropy:', conditional_entropy)

Conditional Entropy: [1.09233492 0.94973535 0.98410866 0.78742796 1.09861229]


In [36]:
#correlation matrix for non categorical variables

#dropping categorical variables
heart=heart.drop(columns=['HighBP','HighChol','CholCheck','Smoker',
                          'Stroke','PhysActivity','Fruits','Veggies',
                          'HvyAlcoholConsump','NoDocbcCost','DiffWalk',
                          'Sex','GenHlth'])

In [37]:
heart.corr().style.background_gradient(cmap="Blues")

,HeartDiseaseorAttack,BMI,Diabetes,AnyHealthcare,MentHlth,PhysHlth,Age,Education,Income
HeartDiseaseorAttack,1.000000,0.052904,0.180272,0.018734,0.064621,0.181698,0.221618,-0.099600,-0.141011
BMI,0.052904,1.000000,0.224379,-0.018471,0.085310,0.121141,-0.036618,-0.103932,-0.100069
Diabetes,0.180272,0.224379,1.000000,0.015410,0.073507,0.176287,0.185026,-0.130517,-0.171483
AnyHealthcare,0.018734,-0.018471,0.015410,1.000000,-0.052707,-0.008276,0.138046,0.122514,0.157999
MentHlth,0.064621,0.085310,0.073507,-0.052707,1.000000,0.353619,-0.092068,-0.101830,-0.209806
PhysHlth,0.181698,0.121141,0.176287,-0.008276,0.353619,1.000000,0.099130,-0.155093,-0.266799
Age,0.221618,-0.036618,0.185026,0.138046,-0.092068,0.099130,1.000000,-0.101901,-0.127775
Education,-0.099600,-0.103932,-0.130517,0.122514,-0.101830,-0.155093,-0.101901,1.000000,0.449106
Income,-0.141011,-0.100069,-0.171483,0.157999,-0.209806,-0.266799,-0.127775,0.449106,1.000000


In [38]:
heart = pd.read_csv("heart.csv")
heart=heart.drop(columns=['HighBP','HighChol','CholCheck','Smoker',
                          'PhysActivity','Fruits','Veggies',
                          'HvyAlcoholConsump','NoDocbcCost','DiffWalk',
                          'Sex','HeartDiseaseorAttack','GenHlth'])

In [39]:
heart.corr().style.background_gradient(cmap="YlOrBr")

,BMI,Stroke,Diabetes,AnyHealthcare,MentHlth,PhysHlth,Age,Education,Income
BMI,1.000000,0.020153,0.224379,-0.018471,0.085310,0.121141,-0.036618,-0.103932,-0.100069
Stroke,0.020153,1.000000,0.107179,0.008776,0.070172,0.148944,0.126974,-0.076009,-0.128599
Diabetes,0.224379,0.107179,1.000000,0.015410,0.073507,0.176287,0.185026,-0.130517,-0.171483
AnyHealthcare,-0.018471,0.008776,0.015410,1.000000,-0.052707,-0.008276,0.138046,0.122514,0.157999
MentHlth,0.085310,0.070172,0.073507,-0.052707,1.000000,0.353619,-0.092068,-0.101830,-0.209806
PhysHlth,0.121141,0.148944,0.176287,-0.008276,0.353619,1.000000,0.099130,-0.155093,-0.266799
Age,-0.036618,0.126974,0.185026,0.138046,-0.092068,0.099130,1.000000,-0.101901,-0.127775
Education,-0.103932,-0.076009,-0.130517,0.122514,-0.101830,-0.155093,-0.101901,1.000000,0.449106
Income,-0.100069,-0.128599,-0.171483,0.157999,-0.209806,-0.266799,-0.127775,0.449106,1.000000
